In [1]:
!pip install telegram-send

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
import pandas as pd

In [3]:
import telegram_send

In [4]:
import requests

In [5]:
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

In [6]:
def telegram_bot_sendtext(bot_message):
    
    bot_token = ''
    bot_chatID = ''
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()

In [7]:
#telegram_bot_sendtext("Heloo World")

In [8]:
from project_lib import Project

In [9]:
project = Project(
    project_id = '', 
    project_access_token = ''
)
pc = project.project_context

In [10]:
#header = ['qid', 'pergunta', 'votos', 'nRespostas','views','tags','descricao','lastModifyed','socreatedAt']
header = ['qid', 'pergunta', 'votos', 'nRespostas','views','tags','lastModifyed','socreatedAt']

In [11]:
def scrap_unanswered_questions(questions_list_page):
    df = pd.DataFrame(columns = header)
    
    questions = questions_list_page.find_all(class_='question-summary')
    q_count = len(questions)
    
    for i in range(q_count):
        id_str = questions[i].get('id')
        id_clean = re.findall('[0-9]+', id_str)[0]
        #print(id_clean)
        pergunta = questions[i].find(class_='question-hyperlink').text
        #print(pergunta)
        n_votos = questions[i].find(class_='vote-count-post').text
        #print(n_votos)
        n_respostas =  questions[i].find(class_='status').text
        n_respostas = re.findall('[0-9]+', n_respostas)[0]
        #print(n_respostas)
        visitas = questions[i].find(class_='views')['title']
        n_visitas  = re.findall('[0-9]+', visitas)[0]
        #print(n_visitas)
        tags = questions[i].find(class_='flex--item').find_all('a', href=True)
        n_tags = []
        for i in range(len(tags)):
            n_tags.append(tags[i].text)
        #print(n_tags)
        last_modification = questions[i].find(class_='relativetime')['title']
        #print(last_modification)
        
        html = urlopen(f"https://pt.stackoverflow.com/questions/{id_clean}")
        question_page = BeautifulSoup(html, 'html.parser')
        html.close()
        time.sleep(10)
        
        asked_ = question_page.find('time')['datetime']
        #print(asked_)
        
        #description = question_page.find(class_='js-post-body').findAll('p')
        #description_p = []
        #for f in range(len(description)):
            #description_p.append(description[f].text)
            
        #description_p = ' '.join(description_p)
        #print(description_p)
        new_row = {
                   'qid': id_clean, 
                   'pergunta': pergunta, 
                   'votos': n_votos, 
                   'nRespostas': n_respostas ,
                   'views': n_visitas ,
                   'tags': n_tags,
                   #'descricao': description_p,
                   'lastModifyed':last_modification,
                   'socreatedAt': asked_
                  }
        #append row to the dataframe
        df = df.append(new_row, ignore_index=True)
        #print("******************************************************")
    
    return df

In [12]:
# @hidden_cell      
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_6337e1afa96b41479fda20829220b743 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_6337e1afa96b41479fda20829220b743 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_6337e1afa96b41479fda20829220b743 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_6337e1afa96b41479fda20829220b743)


paginas_ja_baixadas = 87
a = 1
while(a == 1):
    try:     
        csv_filename = f'data_stackOverflow_questions_page_{i}.csv'
        body = client_6337e1afa96b41479fda20829220b743.get_object(Bucket='',Key=csv_filename)['Body']
        # add missing __iter__ method, so pandas accepts body as file-like object
        if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

        df_data_1 = pd.read_csv(body)
        paginas_ja_baixadas = paginas_ja_baixadas + 1
    except:
        break
        

In [13]:
message = f"🚀 Scraping\n\nReiniciando o download da página {paginas_ja_baixadas}"
telegram_bot_sendtext(message)

{'ok': True,
 'result': {'message_id': 39,
  'from': {'id': 1749057957,
   'is_bot': True,
   'first_name': 'CloudAdvisor',
   'username': 'cloud_captain_bot'},
  'chat': {'id': 772858692,
   'first_name': 'Ana Clara',
   'last_name': 'Correa',
   'username': 'aclaracorrea',
   'type': 'private'},
  'date': 1626799901,
  'text': '🚀 Scraping\n\nReiniciando o download da página 87'}}

In [ ]:
i = paginas_ja_baixadas
while(a == 1):
    
    paginaURL = f"https://pt.stackoverflow.com/unanswered/tagged/?page={i}&tab=votes"
    #print(paginaURL)
    
    if (i%10 == 0):
        message = f"🚀 Scraping\n\nIniciando o download de mais um bloco de 10 páginas\n\nPágina atual: {paginaURL}"
        telegram_bot_sendtext(message)
    
    try:
        html = urlopen(f"https://pt.stackoverflow.com/unanswered/tagged/?page={i}&tab=votes")
        questions_list_page = BeautifulSoup(html, 'html.parser')
        html.close()
    except Exception as e:
        message = f"🚀 Scraping\n\nNão consegui abrir essa página no stackoverflow: {paginaURL}"
        telegram_bot_sendtext(message)
        telegram_bot_sendtext(e)
        time.sleep(20)
        try:
            html = urlopen(f"https://pt.stackoverflow.com/unanswered/tagged/?page={i}&tab=votes")
            questions_list_page = BeautifulSoup(html, 'html.parser')
            html.close()
        except:
            message = f"🚀 Scraping\n\no código parou de rodar"
            telegram_bot_sendtext(message)
            break
    
    df = scrap_unanswered_questions(questions_list_page)
    
    prefix = "data_"
    file_name = f"{prefix}stackOverflow_questions_page_{i}.csv"
    
    try:
        project.save_data(file_name = file_name, data = df.to_csv(index=False, sep=';', encoding="UTF-16LE", header=False),set_project_asset=None, overwrite=True, direct_storage=False)
    except:
        message = "🚀 Scraping\n\nDeu ruim aqui eim, não deu pra salvar o csv. Vem dar uma olhada."
        telegram_bot_sendtext(message)
        time.sleep(10)
        pass

    i += 1
    time.sleep(15) # dormir por 3 segundos